In [ ]:
!pip install pathway bokeh --quiet

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [5]:
df = pd.read_csv('/content/dataset.csv')
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [6]:
df.isna().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [7]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [8]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [9]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,OccupancyRate
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.105719
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.197500
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.384279
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.513402
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.319792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.793617
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,1.000000
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.334489
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.538511


In [10]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "OccupancyRate"]].to_csv("parking_stream.csv", index=False)

In [11]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    OccupancyRate: float

In [12]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [13]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d"),
    OccupancyRate = data.Occupancy / data.Capacity,
    day_id = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber,
)

In [14]:
id_lookup = (
    data_with_time
    .select(pw.this.day_id, pw.this.SystemCodeNumber)
    .groupby(pw.this.day_id)
    .reduce(
        day_id = pw.reducers.min(pw.this.day_id),
        SystemCodeNumber = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

In [15]:
import datetime
reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day_id,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.OccupancyRate),
        occ_min = pw.reducers.min(pw.this.OccupancyRate),
        cap = pw.reducers.max(pw.this.Capacity),
        day_id = pw.reducers.min(pw.this.day_id),
        lot = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

In [16]:
delta_window = reduced.with_columns(
    price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
)

In [ ]:
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")

    fig.circle("t", "price", source=source, size=6, color="red")

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()
viz.servable()

In [ ]:
pw.io.jsonlines.write(
    delta_window.select(
        pw.this.t,
        pw.this.lot,
        pw.this.price
    ),
    "pricing_output.jsonl"
)

In [ ]:
pw.run()

In [20]:
df_price = pd.read_json("pricing_output.jsonl", lines=True)
df_price["t"] = pd.to_datetime(df_price["t"])

In [ ]:
from bokeh.plotting import figure, show, output_notebook , save , output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

output_notebook()

plots = []
lots = sorted(df_price["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df_price[df_price["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, size=5, marker="circle", color="black")
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

show(column(*plots))


In [ ]:
df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"])

plots = []
lots = sorted(df["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, color="black", size=5, marker="circle")
    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

output_file("all_lot_prices.html")
save(column(*plots))

In [ ]:
from bokeh.models import Legend
df = pd.read_json("pricing_output.jsonl", lines=True)

df["t"] = pd.to_datetime(df["t"])
df["lot"] = df["lot"].astype(str)

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

lots = sorted(df["lot"].unique())
for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot]
    source = ColumnDataSource(df_lot)

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.add_layout(fig.legend[0], 'right')
fig.legend.label_text_font_size = "8pt"

output_file("lot_price_plot_from_json.html")
save(fig)